In [1]:
import sys
sys.path.append("../..")
import xarray as xr
import numpy as np
import os
from surgeNN import io
from surgeNN.evaluation import rmse, compute_precision, compute_recall, compute_f1, add_error_metrics_to_prediction_ds
from surgeNN.preprocessing import deseasonalize_da
import gcsfs
fs = gcsfs.GCSFileSystem() #list stores, stripp zarr from filename, load 

2025-09-25 14:08:29.508761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-25 14:08:29.580619: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Evaluates CoDEC simulations in the same way as the NN predictions:

In [2]:
#configure the script
tgs        = ['stavanger-svg-nor-nhs.csv','wick-wic-gbr-bodc.csv','esbjerg-esb-dnk-dmi.csv','immingham-imm-gbr-bodc.csv','den_helder-denhdr-nld-rws.csv', 'fishguard-fis-gbr-bodc.csv',  'brest-822a-fra-uhslc.csv', 'vigo-vigo-esp-ieo.csv',  'alicante_i_outer_harbour-alio-esp-da_mm.csv']
tgnames = ['Stavanger (NOR)','Wick (UK)', 'Esbjerg (DK)','Immingham (UK)','Den Helder (NL)','Fishguard (UK)','Brest (FR)','Vigo (PT)', 'Alicante (SP)']

qnts = np.array([.95,.98,.99,.995]) #quantiles, don't touch

max_timesteps_between_extremes = 3

save_performance = 1

gtsm_path = '/home/jovyan/surgeNN/input/CoDEC_ERA5_at_gesla3_tgs_eu_hourly_anoms.nc'#'/home/jovyan/test_surge_models/input/CoDEC_ERA5_at_gesla3_tgs_eu_hourly_anoms.nc'
out_path = '/home/jovyan/surgeNN/results/gtsm/CoDEC_ERA5_at_gesla3_tgs_eu_performance.nc'#'/home/jovyan/test_surge_models/results/gtsm/performance/CoDEC_ERA5_at_gesla3_tgs_eu_performance.nc'

In [3]:
lstms = io.Output('gs://leap-persistent/timh37/surgeNN_output/nns/performance_modified_v2/lstm') #load LSTM output to get the timestpe indices to evaluate
lstms.open_performance_data(tgs)
lstms.data = lstms.data.sel(max_timesteps_between_extremes=max_timesteps_between_extremes).load()

observed_thresholds = lstms.observed_thresholds()
observed_stds = lstms.observed_stds()

lstms=lstms.data

/srv/conda/envs/notebook/lib/python3.10/site-packages/xarray/backends/api.py:982: RuntimeWarning: Failed to open Zarr store with consolidated metadata, but successfully read with non-consolidated metadata. This is typically much slower for opening a dataset. To silence this warning, consider:
1. Consolidating metadata in this existing store with zarr.consolidate_metadata().
2. Explicitly setting consolidated=False, to avoid trying to read consolidate metadata, or
3. Explicitly setting consolidated=True, to raise an error in this case instead of falling back to try reading non-consolidated metadata.
  datasets = [open_(p, **open_kwargs) for p in paths]


In [4]:
codec = xr.open_dataset(gtsm_path) #anomalies wrt annual means
codec['surge'] = deseasonalize_da(codec['surge']) #remove seasonal cycle (as done from the predictands)
codec = codec.sel(tg=tgs) #select tide gauges
codec = codec.sel(time=np.intersect1d(codec.time,lstms.time)) #select at 3-hourly timesteps of neural network predictions

#compute error metrics:
codec = codec.surge.expand_dims({'split':3},axis=-1).where(lstms.o.isel(it=0))
codec = codec.to_dataset()

where_observed_peaks = (lstms.o.isel(it=0)>=observed_thresholds)

if max_timesteps_between_extremes>0:
    where_observed_peaks = ((where_observed_peaks) & (where_observed_peaks.rolling(time=1+2*int(max_timesteps_between_extremes),center='True').sum()>1)) #from 'compute_statistics_on_output_ds'

codec['rmse_bulk'] = np.sqrt(((lstms.o.isel(it=0) - codec.surge)**2).mean(dim='time'))
codec['r_bulk'] = xr.corr(lstms.o.isel(it=0) ,codec.surge,dim='time')

codec['rmse_extremes'] = np.sqrt(((lstms.o.isel(it=0).where(where_observed_peaks) - codec.surge.where(where_observed_peaks))**2).mean(dim='time'))
codec['r_extremes'] = xr.corr(lstms.o.isel(it=0).where(where_observed_peaks),codec.surge.where(where_observed_peaks),dim='time')

codec_exceedances = codec.surge>=observed_thresholds
observed_exceedances = where_observed_peaks

codec['true_pos'] =  ((where_observed_peaks) & (codec_exceedances)).where(np.isfinite(lstms.o.isel(it=0))).sum(dim='time')
codec['false_neg'] =  ((where_observed_peaks) & ((codec_exceedances)==False)).where(np.isfinite(lstms.o.isel(it=0))).sum(dim='time')
codec['false_pos'] =  (((where_observed_peaks)==False) & (codec_exceedances)).where(np.isfinite(lstms.o.isel(it=0))).sum(dim='time')
codec['true_neg'] =  (((where_observed_peaks)==False) & ((codec_exceedances)==False)).where(np.isfinite(lstms.o.isel(it=0))).sum(dim='time')

#confusion matrix derivatives
codec['precision'] = compute_precision(codec.true_pos,codec.false_pos)
codec['recall'] = compute_recall(codec.true_pos,codec.false_neg)
codec['f1'] = compute_f1(codec.precision,codec.recall)

for metric in ['r_extremes','rmse_extremes','true_pos','false_neg','false_pos','true_neg','precision','recall','f1']:
    codec[metric] = codec[metric].expand_dims(dim='max_timesteps_between_extremes') #add additional dimension for max time distance used between 
codec['max_timesteps_between_extremes'] = [max_timesteps_between_extremes]

if save_performance:
    codec.to_netcdf(out_path,mode='w')